多进程在notebook环境下运行受限，将其写入到python文件然后执行

In [1]:
%%writefile cs.py
# 创建一个子进程
import time
from multiprocessing import Process

def func():
    time.sleep(3)
    print("Child process done!")

if __name__ == "__main__":
    p = Process(target=func)
    p.start()
    time.sleep(2)
    # p.join()
    print("Parent process done!")

Overwriting cs.py


In [2]:
!python cs.py

Parent process done!
Child process done!


In [3]:
%%writefile cs.py
# 利用队列进行进程同步
import time
from multiprocessing import Process, Queue

def func1(q):
    q.put([42, None, 'hello'])
    time.sleep(2)
    q.put(['happy new Monday!'])

if __name__ == "__main__":
    q = Queue()
    p = Process(target=func1, args=(q, ))
    p.start()
    print(q.get())
    print(q.get())
    print("the end.")
    p.join()

Overwriting cs.py


In [4]:
!python cs.py

[42, None, 'hello']
['happy new Monday!']
the end.


In [5]:
%%writefile cs.py
# 通过加锁进行资源同步
import time
import multiprocessing as mp
from multiprocessing import Process, Lock

def f(i, l):
    l.acquire()
    try:
        print("hello, world", i)
        time.sleep(1)
        print(i, "do something")
    finally:
        l.release()
        pass

if __name__ == "__main__":
    mp.set_start_method('fork')
    lock = Lock()
    for num in range(5):
        Process(target=f, args=(num, lock,)).start()

Overwriting cs.py


In [6]:
!python cs.py

hello, world 0
0 do something
hello, world 1
1 do something
hello, world 2
2 do something
hello, world 3
3 do something
hello, world 4
4 do something


In [7]:
%%writefile cs.py
# 使用value和Array共享内存
from multiprocessing import Process, Value, Array
def f1(n, a):
    n.value = 3
    for i in range(len(a)):
        a[i] += 1

def f2(n, a):
    n.value = 1
    for i in range(len(a)):
        a[i] -= 2

if __name__ == "__main__":
    num = Value('d', 0.0)   # 声明两个可共享的对象
    arr = Array('i', range(10)) # 初始化一个数组
    
    p1 = Process(target=f1, args=(num, arr))
    p2 = Process(target=f2, args=(num, arr))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    print(num.value)
    print(arr[:])

Overwriting cs.py


In [8]:
!python cs.py

1.0
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8]


In [9]:
%%writefile cs.py
# 类里的变量被独自继承
from multiprocessing import Process

class MyProcess(Process):   # 继承Process类
    pool = [0, 0]   # 每个进程继承独立的副本
    def __init__(self, n, name):
        super().__init__()  # 如果自己想要传参name，首先用super执行父类的init方法
        self.n = n
        self.name = name 
    def run(self):
        print("子进程的名字是 >>>", self.name)
        if MyProcess.pool[0] == 0:
            MyProcess.pool[0] = self.n
            MyProcess.pool[1] += 1
        else:
            MyProcess.pool[1] = self.n
        print(MyProcess.pool)


if __name__ == "__main__":
    p1 = MyProcess(101, name="child process 1")
    p2 = MyProcess(102, name="child process 2")

    p1.start()
    p2.start()

    p1.join()
    p2.join()
    print("main process done!")

Overwriting cs.py


In [10]:
!python cs.py

子进程的名字是 >>> child process 1
[101, 1]
子进程的名字是 >>> child process 2
[102, 1]
main process done!


In [11]:
# python用迭代器实现协程
def gen_cal():
    x, y = 1, 1
    exp = None
    while x < 256:
        if exp == None: # 这里接受发送的值
            x, y = y, x + y
            exp = yield y
        else:
            exp = yield (eval(exp))
gc = gen_cal()

In [12]:
print(next(gc))     # next其实等价于gc.send(None)
print(next(gc))
print(gc.send('23+9/3.0'))
print(next(gc))

2
3
26.0
5


In [13]:
def consumer():
    r = ""
    while True:
        n = yield r
        if not n:
            return 
        print("[CONSUMER] Consuming %s..." % n)
        r = "200 OK"

def produce(c):
    c.send(None)
    n = 0
    while n < 5:
        n += 1
        print("[PRODUCER] Producing %s..." % n)
        r = c.send(n)
        print("[PRODUCER] Consumer return: %s" % r)
    c.close()

c = consumer()
produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Consumer return: 200 OK


In [14]:
# 任务轮转调度：用协程实现
def task1():
    timeN = 12
    dur = 6
    while timeN > 0:
        timeN -= dur
        print("Task1 need: ", timeN)
        yield timeN     # 中断
    print("Task1 Finished")
    
def task2():
    timeN = 11
    dur = 3
    while timeN > 0:
        timeN -= dur
        print("Task2 need: ", timeN)
        yield           # 只交出运行权，可以不返回值
    print("Task2 Finished")

def RoundRobin(*task):
    tls = list(task)
    while len(tls) > 0:
        for p in tls:
            try:
                next(p)
            except StopIteration:
                tls.remove(p)
    print("All finished!")

t1 = task1()
t2 = task2()

RoundRobin(t1, t2)

Task1 need:  6
Task2 need:  8
Task1 need:  0
Task2 need:  5
Task1 Finished
Task2 need:  2
Task2 need:  -1
Task2 Finished
All finished!


In [15]:
# 回调函数
def demo_callback(st):
    print(st)

def caller(args, func):
    print("Caller: Do something.")
    func(args)

caller(("I am callee"), demo_callback)

Caller: Do something.
I am callee


In [16]:
# 回调函数方案实现函数的功能组合
def demo_handle(func, args, callback):
    result = func(*args)
    callback(result, func.__name__)

def add(x, y):
    return x + y

def notify(result, frm):
    print("Call fun {}() result = {}".format(frm, result))

demo_handle(add, (3, 5), callback=notify)

Call fun add() result = 8


In [17]:
# 异步框架函数，用来协调运行
def apply_handler(func, args, callback):
    result = func(*args)
    callback(result)

def add(x, y):
    return x + y

def times(x, y):
    return x * y

def make_handler():
    counter = 0
    def handler(result):
        nonlocal counter
        counter += 1
        print("counter = {} result: {}".format(counter, result))
    return handler

handler = make_handler()    # 类似一个装饰器的功能，可具备全局管理功能
apply_handler(add, (2, 3), callback=handler)
apply_handler(times, (4, 6), callback=handler)

counter = 1 result: 5
counter = 2 result: 24


In [18]:
# 异步方式组装回调函数
def apply_async(func, args, *, callback):
    result = func(*args)
    callback.send(result)

def add(x, y):
    return x + y

def times(x, y):
    return x * y

def make_handler():
    counter = 0
    while True:
        result = yield
        counter += 1
        print("counter = {} result: {}".format(counter, result))

handler = make_handler()
next(handler)

apply_async(add, (3, 5), callback=handler)
apply_async(times, (3, 5), callback=handler)

counter = 1 result: 8
counter = 2 result: 15


In [19]:
# 轮转调度队列中的协程
waiting_list = []
class Handle(object):   # 对准备调度的任务进行包装，如设置时间片、优先级、最大运行时间
    def __init__(self, gen):
        self.gen = gen
    def call(self):
        next(self.gen)
        waiting_list.append(self)

def RoundRobin(*tasks):
    waiting_list.extend(Handle(c) for c in tasks)
    while waiting_list:
        for p in waiting_list:
            try:
                p.call()
            except StopIteration:
                waiting_list.remove(p)
    
    print("All finished!")

def task1():
    timeN = 12
    dur = 6
    while timeN > 0:
        timeN -= dur
        print("task1 need: ", timeN)
        yield timeN
    print("Task1 finished!")

def task2():
    timeN = 11
    dur = 3
    while timeN > 0:
        timeN -= dur
        print("Task2 need: ", timeN)
        yield

    print("Task2 need: ", timeN)

RoundRobin(task1(), task2())


task1 need:  6
Task2 need:  8
task1 need:  0
Task2 need:  5
Task1 finished!
Task2 need:  2
Task2 need:  -1
Task2 need:  -1
All finished!


In [24]:
# 包装异步任务实现轮转调度框架
waiting_list = []
class Handle(object):
    def __init__(self, gen, pri=0.5):
        self.gen = gen
        self.timeSlice = 0
        self.timeNeed = 0
        self.pri = pri   # 优先级，级别越高运行时间越大
    def call(self):
        try:
            if self.timeSlice == 0:
                self.timeNeed = next(self.gen)
                self.timeSlice = int(self.timeNeed * self.pri)
            else:
                self.gen.send(self.timeSlice)
            
            waiting_list.append(self)
        except StopIteration:
            print(self.gen.__name__, "finished")

def RoundRobin(*tasks):
    waiting_list.extend(Handle(c) for c in tasks)
    while waiting_list:
        p = waiting_list.pop(0)
        p.call()
    print("All finished!")

def task1():
    timeN = 12
    dur = 6
    while timeN > 0:
        timeN -= dur
        print("task1 need: ", timeN)
        yield timeN
    print("Task1 finished!")

def task2():
    timeN = 11
    dur = 3
    while timeN > 0:
        timeN -= dur
        print("Task2 need: ", timeN)
        yield timeN

    print("Task2 need: ", timeN)
    
RoundRobin(task1(), task2())

task1 need:  6
6
Task2 need:  8
8
task1 need:  0
Task2 need:  5
Task1 finished!
task1 finished
Task2 need:  2
Task2 need:  -1
Task2 need:  -1
task2 finished
All finished!


In [26]:
%%writefile cs.py
# python定义协程
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")
    await say_after(1, 'hello')
    await say_after(2, 'world')
    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

Overwriting cs.py


In [27]:
!python cs.py

started at 14:17:49
hello
world
finished at 14:17:52


In [35]:
%%writefile server.py
# 单线程服务端
import socket
import time

# define server info
print("initialize server info")
port = 5003
host = '0.0.0.0'
address = (host, port)

# create TCP connection -- socket
print("create TCP connection -- socket...")
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# bind local machine
print("bind local machine...")
server.bind(address)

# start server
print("start server...")
server.listen(5)

# wait for connection
while True:
    print("wait for client to connect")
    conn, addr = server.accept()
    print(f"the connected client socket object: {conn}\nClient IP address: {addr}")
    # send data
    conn.send("Welcome to the server".encode('utf-8'))
    time.sleep(100)
    conn.close()

Overwriting server.py


In [ ]:
%%writefile client.py
import socket
port = 5003
hostname = '127.0.0.1'
client = socket.socket()
client.connect((hostname, port))
data = client.recv(100).decode('utf-8')
print(data)

client.close()

Overwriting client.py


In [37]:
!python server.py

initialize server info
create TCP connection -- socket...
bind local machine...
start server...
wait for client to connect
^C
Traceback (most recent call last):
  File "/Users/ceoxnim/Documents/university study/22春python课程/slideCodeRevise/server.py", line 26, in <module>
    conn, addr = server.accept()
  File "/Users/ceoxnim/miniforge3/envs/py39/lib/python3.9/socket.py", line 293, in accept
    fd, addr = self._accept()
KeyboardInterrupt


In [ ]:
!python client.py